**IMPORT PYNGROK TO RUN STREAMLIT ON GOOGLE COLAB**

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


**CREATING APP.PY FILE TO RUN STREAMLIT IN LOCAL SERVER**

In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
import pickle
import pandas as pd
from datetime import date

# === App Config ===
st.set_page_config(page_title="Rossmann Sales Forecast", layout="centered")
st.markdown("<h1 style='text-align: center; color: #b30000;'>ROSSMANN</h1>", unsafe_allow_html=True)
st.markdown("<h3 style='text-align: center;'>Smart Sales Forecasting</h3>", unsafe_allow_html=True)

# === Load Model ===
@st.cache_resource
def load_model(path):
    try:
        with open(path, 'rb') as file:
            return pickle.load(file)
    except Exception as e:
        st.error(f" Failed to load model: {e}")
        return None

MODEL_PATH = "RossmannStore.pkl"
model = load_model(MODEL_PATH)

# === Form UI ===
with st.form("prediction_form"):
    st.markdown("###  Select Date Range")
    col1, col2 = st.columns(2)
    with col1:
        from_date = st.date_input("From:", value=date(2015, 8, 1))
    with col2:
        to_date = st.date_input("To:", value=date(2015, 8, 31))

    stores = st.text_input("Stores (comma separated):", value="1113,1114,1115")

    st.markdown("###  Store Settings")
    day_of_week = st.selectbox("Day of Week", options=[1, 2, 3, 4, 5, 6, 7], index=4)
    open_status = st.radio("Is Store Open?", ["Yes", "No"]) == "Yes"
    promo_active = st.radio("Promo Active?", ["Yes", "No"]) == "Yes"
    state_holiday = st.radio("Is it a State Holiday?", ["No", "Yes"]) == "Yes"
    school_holiday = st.radio("Is it a School Holiday?", ["No", "Yes"]) == "Yes"
    customers = st.slider("Expected Customers", min_value=0, max_value=5000, value=600)

    submitted = st.form_submit_button("🔍 Predict Sales")

# === Prediction Logic ===
if submitted:
    if model is not None:
        try:
            store_ids = [int(s.strip()) for s in stores.split(",") if s.strip().isdigit()]
            predictions = []

            for store_id in store_ids:
                # Build input feature array
                features = np.array([[
                    store_id,
                    day_of_week,
                    int(open_status),
                    int(promo_active),
                    int(state_holiday),
                    int(school_holiday),
                    customers,
                    0, 0, 0, 0, 0, 0, 0, 0, 0, 0  # placeholder values (total 17 features)
                ]], dtype=np.float32)

                pred = model.predict(features)
                predicted_value = float(pred[0])
                predictions.append((store_id, round(predicted_value, 2)))

            st.markdown("### 📈 Predicted Sales")
            st.dataframe(pd.DataFrame(predictions, columns=["Store ID", "Predicted Sales (€)"]))

        except Exception as e:
            st.error(f" Error during prediction: {e}")
    else:
        st.error(" Model not loaded. Please check if the `.pkl` file is in the correct path.")


Writing app.py


**Getting authenticatae token for ngrok and define a funtion to make UI interface in local server **

In [3]:
import threading
import time
from pyngrok import ngrok
ngrok.set_auth_token("2ySL0saJYSgzKIcg1NNACGs2gXP_4SCQYsLJSQ6d5Edt8M3v6")

def run():
    !streamlit run app.py &

# Start Streamlit in a separate thread
thread = threading.Thread(target=run)
thread.start()

# Wait a little for the server to start
time.sleep(3)

# Get public URL
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.234.28.138:8501

Streamlit app is live at: NgrokTunnel: "https://0849-35-234-28-138.ngrok-free.app" -> "http://localhost:8501"
